In [1]:
%pip install -U langchain langchain-openai

from langchain.prompts import PromptTemplate  # LangChain의 프롬프트 템플릿 모듈
from langchain_openai import ChatOpenAI  # OpenAI의 챗봇 모델을 사용하기 위한 모듈
from langchain.schema import StrOutputParser  # 모델의 출력을 문자열로 변환하는 모듈
from langchain.schema.runnable import RunnablePassthrough  # 데이터 변환 없이 그대로 전달하는 모듈
import os  # 환경 변수 설정을 위한 모듈

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from dotenv import load_dotenv

load_dotenv()  # .env 파일 로드
api_key = os.getenv("OPENAI_API_KEY")


In [ ]:
# 여행 가이드 챗봇 프롬프트 템플릿 설정
tour_guide_template = """
당신은 여행을 좋아하는 10년차 여행 가이드입니다.
사용자가 입력한 여행 국가와 도시에 맞춰 3일간의 가상 여행 계획을 작성해주세요.
반드시 아래 FORMAT에 맞춰 작성하세요.

#상황:
{question}

#FORMAT:
- Day1: 
- Day2: 
- Day3: 
"""

In [ ]:
# 사용자로부터 여행 국가와 도시를 입력받음
country = input("여행 국가를 입력하세요: ")  # 예: "프랑스"
city = input("여행 도시를 입력하세요: ")  # 예: "파리"

In [ ]:
# 입력받은 국가와 도시를 하나의 문자열로 구성
# 입력값을 포맷에 맞춰 문자열로 변환
user_input = f"여행 국가: {country}, 여행 도시: {city}"  # "여행 국가: <국가>, 여행 도시: <도시>" 형식으로 만들어야 함

In [ ]:
# LangChain의 프롬프트 템플릿을 적용하여 사용자 입력을 프롬프트에 반영
prompt = PromptTemplate.from_template(user_input)  # 프롬프트 템플릿을 이용하여 객체 생성

In [ ]:
# OpenAI의 GPT-4o-mini 모델을 활용하여 여행 일정 생성
model = ChatOpenAI(model_name="gpt-5-nano")  # OpenAI의 GPT-4o-mini 모델 사용 (gpt-5-nano 모델 사용 가능한 api key 였음)

In [ ]:
# 모델의 출력을 문자열로 변환하는 파서 (결과 데이터를 가공 없이 그대로 출력할 수 있도록 설정)
output_parser = StrOutputParser()  # 모델 결과를 문자열로 변환하는 파서

In [ ]:
# 체인 구성: 
# 1. 사용자 입력값을 그대로 전달 (RunnablePassthrough)
# 2. 프롬프트 템플릿 적용
# 3. OpenAI 모델을 실행하여 답변 생성
# 4. 생성된 답변을 문자열로 변환하여 반환
# 체인 구성: 사용자 입력 → 변환(RunnablePassthrough) → 프롬프트 적용 → 모델 실행 → 출력 변환
chain = { "question": RunnablePassthrough() } | prompt | model | output_parser

In [ ]:
# 체인을 실행하여 여행 계획을 생성
# 사용자가 입력한 국가와 도시 정보를 `question` 키에 매핑하여 전달
answer = chain.invoke({"question": user_input})  # 체인을 실행하고 결과를 가져옴

In [ ]:
# 생성된 여행 계획 출력
print("\n[여행 계획]")  # 여행 계획 제목 출력
print(answer)  # 모델이 생성한 여행 계획 출력


[여행 계획]
멋진 선택이네요! 파리 여행 plan을 도와드리겠습니다. 원하시는 일정에 맞춰 맞춤형 계획을 만들어 드리려면, 몇 가지를 알려주시면 좋습니다. 우선 기본적인 아이디어를 몇 가지 공유하고, 원하시면 구체적인 일정으로 바로 작성해 드릴게요.

간단한 코스 아이디어 (시간 여유가 있다면 참고)
- 3일 코스
  - Day 1: 루브르 미술관 방문(오전), 튀일리 정원 산책, 센 강 크루즈로 하루 마무리
  - Day 2: 에펠탑 오전 방문 → 샹젤리제 거리와 개선문 구경 → 몽마르트 언덕의 사크레쾨르 대성당 방문
  - Day 3: Île de la Cité의 노트르담 외관/생탈샤펠르(세인트샤펠르) 방문 → 마레 지구 산책 및 빌드거리 카페 탐방
- 5일 코스
  - 앞의 3일에 더해 Day 4: 베르사유 궁전 당일치기(대저택과 정원) 또는 기차로 기수 지역 당일치기
  - Day 5: 오르세 미술관 및 로댕 미술관 방문, Luxembourg Gardens 산책
- 7일 코스(느긋하게, 깊이 있게)
  - 위에 5일 코스에 더해 Latin Quarter, Saint-Germain-des-Prés, Canal Saint-Martin 산책
  - 또 한 번의 박물관 집중: Centre Pompidou, Musée Picasso 또는 Musée Carnavalet 방문
  - 하루를 몽생미샤르 지역이나 몽파르나스 지구 탐방으로 마무리
  - 가능하다면 샤모니/샤르트르 같은 당일치기(기차)으로 확장

주요 팁
- 교통: 파리 시내는 지하철(Métro)과 RER로 이동이 편리합니다. 4일 이상 체류 시 Navigo 주간 패스나 Paris Visite 패스를 고려해 보세요.
- 입장 예약: 루브르, 오르세, 에펠탑 등은 사전 티켓 예약이 좋습니다. 특히 에펠탑은 대기시간이 길 수 있어 미리 예매 권장.
- 무료/할인 정보: 유럽 연합 국적의 26세 이하 방문자 등 특정 조건에서 무료/저렴한 입장이 가능한 경우가 있습니다. 또한 일부 박물관은 특정 요일에 무료 